colab notebook

In [1]:
!pip install catboost
!pip install shap
!pip install flaml
!pip install optuna
!pip install mlxtend

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.9/313.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import json
import os, sys
from tqdm import tqdm
import numpy as np
import polars as pl
from polars import col as c
from pprint import pprint
import matplotlib.pyplot as plt
import re
import shap
from flaml.automl import AutoML

sys.path.append("../")
pl.Config.set_tbl_rows(50)

# explicitly require this experimental feature
from sklearn.experimental import enable_iterative_imputer  # noqa

# now you can import normally from sklearn.impute
from sklearn.impute import IterativeImputer
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor, CatBoostClassifier

In [4]:
FILES_DIR = "./"
### select data
path = "./"
os.makedirs(os.path.join(FILES_DIR, path), exist_ok=True)
file_name = "imputed_train_data.parquet"
file_path = os.path.join(FILES_DIR, path, file_name)
# train_data.write_parquet(file_path)
train_data = pl.read_parquet(file_path)
### split data
X_train, X_test, y_train, y_test = train_test_split(
    train_data.select(
        pl.selectors.by_dtype(pl.Float64, pl.Int64, pl.Int32),
    ).drop("target"),
    train_data["target"],
    test_size=0.2,
    random_state=1,
)

X_train, X_eval, y_train, y_eval = train_test_split(
    X_train,
    y_train,
    test_size=0.25,
    random_state=1,
)  # 0.25 x 0.8 = 0.2
cat_cols = X_train.select(
    pl.selectors.by_dtype(pl.String),
).columns

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [10]:
baseline_model = AutoML()

settings = {
    "metric": "roc_auc",
    "estimator_list": ["lgbm", "catboost", "rf"],  # list of ML learners
    "task": "classification",  # task type
    # "log_file_name": "/content/drive/MyDrive/1231_ABCD/baseline_model.log",  # flaml log file
    "time_budget": 200,
    "early_stop": True,
    "eval_method": "cv",
    "n_splits": 5,
    "verbose": 2,
    "seed": 0,
    "split_type": "stratified",
}

In [11]:
baseline_model.fit(
    X_train=X_train.to_pandas(),
    y_train=y_train.to_pandas(),
    **settings,
)

In [12]:
metric = roc_auc_score(
    y_test,
    baseline_model.predict_proba(X_test.to_pandas())[:, 1],
)
print(metric.round(3))

0.648
